## Importacion de librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

Se hace la lectura de la base ya lista para modelar en el proceso anterior

In [2]:
ml_data = pd.read_csv('D:\Proyecto 2\Data\Processed\ml_dataset.csv')
ml_data.head()

,user_id,transaction_amount,time_of_transaction,previous_fraudulent_transactions,account_age,number_of_transactions_last_24h,fraudulent,fraud_per_month,transaction_type_ATM Withdrawal,transaction_type_Bank Transfer,...,location_New York,location_San Francisco,location_Seattle,location_Unknown,payment_method_Credit Card,payment_method_Debit Card,payment_method_Invalid Method,payment_method_Net Banking,payment_method_UPI,payment_method_Unknown
0,4174,1292.76,16.00000,0,119,13,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
1,4507,1554.58,13.00000,4,79,3,0,20,1,0,...,1,0,0,0,1,0,0,0,0,0
2,1860,2395.02,11.48752,3,115,9,0,38,1,0,...,0,0,0,1,0,0,0,0,0,1
3,2294,100.10,15.00000,4,3,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2130,1490.50,19.00000,2,57,7,0,28,0,0,...,0,1,0,0,1,0,0,0,0,0


In [3]:
ml_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51000 entries, 0 to 50999
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   user_id                           51000 non-null  int64  
 1   transaction_amount                51000 non-null  float64
 2   time_of_transaction               51000 non-null  float64
 3   previous_fraudulent_transactions  51000 non-null  int64  
 4   account_age                       51000 non-null  int64  
 5   number_of_transactions_last_24h   51000 non-null  int64  
 6   fraudulent                        51000 non-null  int64  
 7   fraud_per_month                   51000 non-null  int64  
 8   transaction_type_ATM Withdrawal   51000 non-null  int64  
 9   transaction_type_Bank Transfer    51000 non-null  int64  
 10  transaction_type_Bill Payment     51000 non-null  int64  
 11  transaction_type_Online Purchase  51000 non-null  int64  
 12  tran

In [4]:
ml_data['fraudulent'].value_counts()

0    48490
1     2510
Name: fraudulent, dtype: int64

Debido a la desbalance de clases, se utilizara el algoritmo SMOTE para balancear las clases.

In [5]:
scaler = StandardScaler()
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(ml_data.drop(['fraudulent'], axis=1), ml_data['fraudulent'])
X_scaled = scaler.fit_transform(X_train_resampled)

Se hace el particionamiento de datos

In [6]:
X_entrenamiento_caracteristica, X_prueba_caracteristica, Y_entrenamiento_objetivo, Y_prueba_objetivo = train_test_split(X_scaled, y_train_resampled, random_state=55, stratify=y_train_resampled, test_size=0.2)

### Comenzar modelado con diferentes opciones

In [7]:
reg_log = LogisticRegression(C=1.0, solver="liblinear", max_iter=500, class_weight="balanced", random_state=4525)

In [8]:
reg_log.fit(X_train_resampled, y_train_resampled)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=4525,
                   solver='liblinear')

In [9]:
Y_pred = reg_log.predict(X_prueba_caracteristica)


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [10]:
accuracy_rl = accuracy_score(Y_prueba_objetivo, Y_pred)
precision_rl = precision_score(Y_prueba_objetivo, Y_pred, zero_division=0)
recall_rl = recall_score(Y_prueba_objetivo, Y_pred)
f1_rl = f1_score(Y_prueba_objetivo, Y_pred)
auc_roc_rl = roc_auc_score(Y_prueba_objetivo, Y_pred)

# Mostrar las métricas
print(f"Accuracy: {accuracy_rl:.4f}")
print(f"Precisión: {precision_rl:.4f}")
print(f"Recall: {recall_rl:.4f}")
print(f"F1-score: {f1_rl:.4f}")
print(f"AUC-ROC: {auc_roc_rl:.4f}")

Accuracy: 0.9704
Precisión: 1.0000
Recall: 0.9407
F1-score: 0.9694
AUC-ROC: 0.9704


In [11]:
resultados_lr = []
resultados_lr.append({
    'Modelo': 'Regresión Logística',
    'Accuracy': accuracy_rl,
    'Precisión': precision_rl,
    'Recall': recall_rl,
    'F1-score': f1_rl,
    'AUC-ROC': auc_roc_rl
})
df_rl = pd.DataFrame(resultados_lr)
df_rl

,Modelo,Accuracy,Precisión,Recall,F1-score,AUC-ROC
0,Regresión Logística,0.970355,1.0,0.940709,0.969449,0.970355


In [12]:
for profundidad in range(1, 15):
    for split in [2, 5, 10]:
        arbol_clf = DecisionTreeClassifier(max_depth=profundidad, min_samples_split=split, criterion="gini", class_weight="balanced", random_state=42)
        arbol_clf.fit(X_train_resampled, y_train_resampled)
        Y_pred = arbol_clf.predict(X_prueba_caracteristica)

        accuracy_arbol = accuracy_score(Y_prueba_objetivo, Y_pred)
        precision_arbol = precision_score(Y_prueba_objetivo, Y_pred, zero_division=0)
        recall_arbol = recall_score(Y_prueba_objetivo, Y_pred)
        f1_arbol = f1_score(Y_prueba_objetivo, Y_pred)
        auc_roc_arbol = roc_auc_score(Y_prueba_objetivo, Y_pred)

        print(f"Profundidad: {profundidad}, Split: {split}, Accuracy: {accuracy_arbol:.4f}, Precisión: {precision_arbol:.4f}, Recall: {recall_arbol:.4f}, F1-score: {f1_arbol:.4f}, AUC-ROC: {auc_roc_arbol:.4f}")


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Profundidad: 1, Split: 2, Accuracy: 0.6066, Precisión: 0.5666, Recall: 0.9065, F1-score: 0.6973, AUC-ROC: 0.6066
Profundidad: 1, Split: 5, Accuracy: 0.6066, Precisión: 0.5666, Recall: 0.9065, F1-score: 0.6973, AUC-ROC: 0.6066


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Profundidad: 1, Split: 10, Accuracy: 0.6066, Precisión: 0.5666, Recall: 0.9065, F1-score: 0.6973, AUC-ROC: 0.6066


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


...

d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Profundidad: 14, Split: 2, Accuracy: 0.8008, Precisión: 0.7373, Recall: 0.9344, F1-score: 0.8243, AUC-ROC: 0.8008


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Profundidad: 14, Split: 5, Accuracy: 0.8668, Precisión: 0.8346, Recall: 0.9150, F1-score: 0.8730, AUC-ROC: 0.8668
Profundidad: 14, Split: 10, Accuracy: 0.8686, Precisión: 0.8374, Recall: 0.9149, F1-score: 0.8744, AUC-ROC: 0.8686


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


El mejor resultado para el modelo de árboles de decision es el siguiente: 
Profundidad: 7, Split: 10
Accuracy: 0.9244
Precisión: 0.9611
Recall: 0.8845 
F1-score: 0.9212
AUC-ROC: 0.9244

In [13]:
resultados_decision = []

# Agregar la combinación específica al DataFrame
resultados_decision.append({
    'Modelo': 'Decision Tree',
    "Profundidad": 14,
    "Split": 10,
    "Accuracy": 0.8686,
    "Precision": 0.8374,
    "Recall": 0.9149,
    "F1-Score": 0.8744,
    "AUC-ROC": 0.8686
})

# Convertir la lista de resultados en un DataFrame
df_resultados_decision_tree = pd.DataFrame(resultados_decision)

df_resultados_decision_tree

,Modelo,Profundidad,Split,Accuracy,Precision,Recall,F1-Score,AUC-ROC
0,Decision Tree,14,10,0.8686,0.8374,0.9149,0.8744,0.8686


In [14]:
for estimadores in range(10, 101, 10):
    for profundidad in [5, 10, 15]:
        bosque_clf = RandomForestClassifier(n_estimators=estimadores, max_depth=profundidad, min_samples_split=5, class_weight="balanced_subsample", random_state=42)
        bosque_clf.fit(X_train_resampled, y_train_resampled)
        Y_pred = bosque_clf.predict(X_prueba_caracteristica)

        accuracy_bosque = accuracy_score(Y_prueba_objetivo, Y_pred)
        precision_bosque = precision_score(Y_prueba_objetivo, Y_pred, zero_division=0)
        recall_bosque = recall_score(Y_prueba_objetivo, Y_pred)
        f1_bosque = f1_score(Y_prueba_objetivo, Y_pred)
        auc_roc_bosque = roc_auc_score(Y_prueba_objetivo, Y_pred)

        print(f"Estimadores: {estimadores}, Profundidad: {profundidad}, Accuracy: {accuracy_bosque:.4f}, Precisión: {precision_bosque:.4f}, Recall: {recall_bosque:.4f}, F1-score: {f1_bosque:.4f}, AUC-ROC: {auc_roc_bosque:.4f}")

d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Estimadores: 10, Profundidad: 5, Accuracy: 0.8959, Precisión: 0.9262, Recall: 0.8603, F1-score: 0.8920, AUC-ROC: 0.8959


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Estimadores: 10, Profundidad: 10, Accuracy: 0.9248, Precisión: 0.9727, Recall: 0.8742, F1-score: 0.9208, AUC-ROC: 0.9248


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


...

Estimadores: 100, Profundidad: 5, Accuracy: 0.9104, Precisión: 0.9692, Recall: 0.8477, F1-score: 0.9044, AUC-ROC: 0.9104


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Estimadores: 100, Profundidad: 10, Accuracy: 0.9338, Precisión: 0.9989, Recall: 0.8685, F1-score: 0.9292, AUC-ROC: 0.9338


d:\Anaconda\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Estimadores: 100, Profundidad: 15, Accuracy: 0.9498, Precisión: 0.9998, Recall: 0.8999, F1-score: 0.9472, AUC-ROC: 0.9498


El mejor resultado para Random Forest es con los siguientes estimadores:
Estimadores: 100, Profundidad: 15
Accuracy: 0.9498
Precisión: 0.9998
Recall: 0.8999 
F1-score: 0.9472
AUC-ROC: 0.9498

In [15]:
resultados_random_forest = []

# Agregar la combinación específica al DataFrame
resultados_random_forest.append({
    'Modelo': 'Random Forest',
    "Estimadores": 100,
    "Profundidad": 15,
    "Accuracy": 0.9498,
    "Precision": 0.9998,
    "Recall": 0.8999,
    "F1-Score": 0.9472,
    "AUC-ROC": 0.9498
})

# Convertir la lista de resultados en un DataFrame
df_resultados_random_forest = pd.DataFrame(resultados_random_forest)

df_resultados_random_forest

,Modelo,Estimadores,Profundidad,Accuracy,Precision,Recall,F1-Score,AUC-ROC
0,Random Forest,100,15,0.9498,0.9998,0.8999,0.9472,0.9498


## Test

In [16]:
#Regresion logistica
reg_log.fit(X_prueba_caracteristica, Y_prueba_objetivo)
Y_pred_test = reg_log.predict(X_prueba_caracteristica)

accuracy_rl_test = accuracy_score(Y_prueba_objetivo, Y_pred_test)
precision_rl_test = precision_score(Y_prueba_objetivo, Y_pred_test, zero_division=0)
recall_rl_test = recall_score(Y_prueba_objetivo, Y_pred_test)
f1_rl_test = f1_score(Y_prueba_objetivo, Y_pred_test)
auc_roc_rl_test = roc_auc_score(Y_prueba_objetivo, Y_pred_test)

resultados_lr_test = []
resultados_lr_test.append({
    'Modelo': 'Regresión Logística',
    'Accuracy': accuracy_rl_test,
    'Precisión': precision_rl_test,
    'Recall': recall_rl_test,
    'F1-score': f1_rl_test,
    'AUC-ROC': auc_roc_rl_test
})
df_rl_test = pd.DataFrame(resultados_lr_test)
df_rl_test

,Modelo,Accuracy,Precisión,Recall,F1-score,AUC-ROC
0,Regresión Logística,0.970767,1.0,0.941534,0.969887,0.970767


In [20]:
#Arbo de decision 
arbol_clf = DecisionTreeClassifier(max_depth=7, min_samples_split=10, criterion="gini", class_weight="balanced", random_state=42)
arbol_clf.fit(X_prueba_caracteristica, Y_prueba_objetivo)
Y_pred_test = arbol_clf.predict(X_prueba_caracteristica)

accuracy_bosque_test = accuracy_score(Y_prueba_objetivo, Y_pred_test)
precision_bosque_test = precision_score(Y_prueba_objetivo, Y_pred_test, zero_division=0)
recall_bosque_test = recall_score(Y_prueba_objetivo, Y_pred_test)
f1_bosque_test = f1_score(Y_prueba_objetivo, Y_pred_test)
auc_roc_bosque_test = roc_auc_score(Y_prueba_objetivo, Y_pred_test)

resultados_decision_test = []

# Agregar la combinación específica al DataFrame
resultados_decision_test.append({
    'Modelo': 'Decision Tree',
    "Profundidad": 14,
    "Split": 10,
    "Accuracy": accuracy_bosque_test,
    "Precision": precision_bosque_test,
    "Recall": recall_bosque_test,
    "F1-Score": f1_bosque_test,
    "AUC-ROC": auc_roc_bosque_test
})

# Convertir la lista de resultados en un DataFrame
df_resultados_decision_tree_test = pd.DataFrame(resultados_decision_test)

df_resultados_decision_tree_test

,Modelo,Profundidad,Split,Accuracy,Precision,Recall,F1-Score,AUC-ROC
0,Decision Tree,14,10,0.92782,0.959876,0.892968,0.925214,0.92782


In [18]:
#Random Forest
bosque_clf = RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_split=5, class_weight="balanced_subsample", random_state=42)
bosque_clf.fit(X_prueba_caracteristica, Y_prueba_objetivo)
Y_pred_test = bosque_clf.predict(X_prueba_caracteristica)

accuracy_bosque_test = accuracy_score(Y_prueba_objetivo, Y_pred_test)
precision_bosque_test = precision_score(Y_prueba_objetivo, Y_pred_test, zero_division=0)
recall_bosque_test = recall_score(Y_prueba_objetivo, Y_pred_test)
f1_bosque_test = f1_score(Y_prueba_objetivo, Y_pred_test)
auc_roc_bosque_test = roc_auc_score(Y_prueba_objetivo, Y_pred_test)

resultados_random_forest_test = []

# Agregar la combinación específica al DataFrame
resultados_random_forest_test.append({
    'Modelo': 'Random Forest',
    "Estimadores": 100,
    "Profundidad": 15,
    "Accuracy": accuracy_bosque_test,
    "Precision": precision_bosque_test,
    "Recall": recall_bosque_test,
    "F1-Score": f1_bosque_test,
    "AUC-ROC": auc_roc_bosque_test
})

# Convertir la lista de resultados en un DataFrame
df_resultados_random_forest_test = pd.DataFrame(resultados_random_forest_test)

df_resultados_random_forest_test

,Modelo,Estimadores,Profundidad,Accuracy,Precision,Recall,F1-Score,AUC-ROC
0,Random Forest,100,15,0.975407,1.0,0.950815,0.974787,0.975407
